In [1]:
import sys
import json
import torch
from pathlib import Path
import tensorboardX
from typing import List
import plotly.express as px
import plotly.graph_objects as go

# import locals
sys.path.insert(0, str(Path('..').resolve()))
from tlaloc import StockDataModule, SeqDataset
from tlaloc import StockGRUModel
%matplotlib inline

In [2]:
trnsf_path = Path('../output/models/params.json').resolve()
model_path = Path('../output/models/model.pth').resolve()

behind = 100
forward = 5

In [3]:
with open(trnsf_path, 'r') as f:
    meta = json.load(f)
print(json.dumps(meta, indent=4))

{
    "stock": "MSFT",
    "model": {
        "input_dim": 1,
        "hidden_dim": 32,
        "num_layers": 2,
        "output_dim": 1,
        "lr": 0.01
    },
    "data": {
        "min": 48.43,
        "max": 260.73999,
        "window": 100
    }
}


In [4]:
model = StockGRUModel(**meta['model'])
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [5]:
msft = StockDataModule.get_stock_data('../data', meta['stock'])
seq = torch.FloatTensor(msft['Close'].values)[-behind*2:]
seq_adj = SeqDataset.scale(seq, meta['data']['min'], meta['data']['max'])

In [6]:
#import torch, gc
#gc.collect()
#torch.cuda.empty_cache()

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [8]:
import numpy as np
seq_test = torch.FloatTensor(np.array([i for i in range(10)]))
print(seq_test)
win, lookahead = 3, 4


overlap = []

for i in reversed(range(1, lookahead+1)):
    print(i, seq_test[-win-i:-i])
    overlap.append(float(i))

print('overlap:', overlap)

test_inputs = list(seq_test[-win:].numpy())
print('prediction:', test_inputs)
for i in range(lookahead):
    print(i, test_inputs[-win:])
    test_inputs.append(float(i))

print(test_inputs)
print(test_inputs[-lookahead:])


tensor([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])
4 tensor([3., 4., 5.])
3 tensor([4., 5., 6.])
2 tensor([5., 6., 7.])
1 tensor([6., 7., 8.])
overlap: [4.0, 3.0, 2.0, 1.0]
prediction: [7.0, 8.0, 9.0]
0 [7.0, 8.0, 9.0]
1 [8.0, 9.0, 0.0]
2 [9.0, 0.0, 1.0]
3 [0.0, 1.0, 2.0]
[7.0, 8.0, 9.0, 0.0, 1.0, 2.0, 3.0]
[0.0, 1.0, 2.0, 3.0]


In [9]:

def predict(seq: torch.Tensor, window: int, lookbehind: int, lookahead: int) -> List:
    overlap = []
    predixn = list(seq[-window:].numpy())
    with torch.no_grad():
        model.to(device)
        model.eval()
        # get previous lookahead period (overlap)
        for i in reversed(range(1, lookbehind+1)):
            x = torch.FloatTensor(seq[-window-i:-i]).view(1, -1)
            y = model(x.to(device))
            overlap.append(y.detach().cpu().item())

        # begin future lookahead period (predictions)
        for i in range(lookahead):
            x = torch.FloatTensor(predixn[-window:]).view(1, -1)
            y = model(x.to(device))
            predixn.append(y.detach().cpu().item())

    return overlap + predixn[-lookahead:]

In [10]:

p_seq = predict(seq_adj, meta['data']['window'], behind, forward)

In [11]:
seq_size = seq_adj.size(0)
actual_range = [i for i in range(seq_size)]
pred_range = [i for i in range(seq_size-behind,seq_size+forward)]
print(seq_size, seq_size-behind, seq_size+forward)

200 100 205


In [12]:
dmin, dmax = meta['data']['min'], meta['data']['max']
actual = SeqDataset.inverse_scale(seq_adj, dmin, dmax)
predicted = SeqDataset.inverse_scale(torch.FloatTensor(p_seq), dmin, dmax)
fig = go.Figure()
fig.add_trace(go.Scatter(x=actual_range, y=actual.numpy(), mode='lines', name='Actual'))
fig.add_trace(go.Scatter(x=pred_range, y=predicted.numpy(), mode='lines', name='Prediction'))
fig.update_layout(title=f'Stock Prediction ({meta["stock"]})',
                   xaxis_title='Day',
                   yaxis_title='Scaled Price')
fig.show()